In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [ ]:
from constant import FeatureType
from loader import Power, Weather

import argparse

### Experiment

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = Path(os.getcwd())
args.experiment_name = "LSTM-001"
args.automl_name = 'automl-001'

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

power = Power(args)
weather = Weather(args, features)

power_data = power.get_data()
model_manager = Model(power_data['scaler'])
setattr(args, 'test_start', power.test_start)
setattr(args, 'test_end', power.test_end)

y_pred_list = []
results = []
for i in range(len(features)):
    weather_data = weather.get_data(i + 1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'feature-%02d' % (i + 1))

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}
    args = model_manager.read_setting(args)
    model = model_manager.train(args, dataset)
    y_pred = model.predict(dataset['test'][0])
    y_pred_list.append(y_pred)
    result = model_manager.evaluate(model, dataset['test'][0], dataset['test'][1])
    results.append(result)

setattr(args, 'features', features)
model_manager.save_result(dataset['test'][1], y_pred_list, args)